In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Data Preprocessing

In [3]:
train=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
stores=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
transactions=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
oil=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
holidays=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")

print(train.head())

   id        date  store_nbr      family  sales  onpromotion
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0
1   1  2013-01-01          1   BABY CARE    0.0            0
2   2  2013-01-01          1      BEAUTY    0.0            0
3   3  2013-01-01          1   BEVERAGES    0.0            0
4   4  2013-01-01          1       BOOKS    0.0            0


In [4]:
print(stores.head())

   store_nbr           city                           state type  cluster
0          1          Quito                       Pichincha    D       13
1          2          Quito                       Pichincha    D       13
2          3          Quito                       Pichincha    D        8
3          4          Quito                       Pichincha    D        9
4          5  Santo Domingo  Santo Domingo de los Tsachilas    D        4


In [5]:
print(transactions.head())
print(transactions.info())

         date  store_nbr  transactions
0  2013-01-01         25           770
1  2013-01-02          1          2111
2  2013-01-02          2          2358
3  2013-01-02          3          3487
4  2013-01-02          4          1922
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB
None


In [6]:
print(oil.head(), oil.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB
         date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20 None


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [7]:
print(holidays.head(), holidays.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB
         date     type    locale locale_name                    description  \
0  2012-03-02  Holiday     Local       Manta             Fundacion de Manta   
1  2012-04-01  Holiday  Regional    Cotopaxi  Provincializacion de Cotopaxi   
2  2012-04-12  Holiday     Local      Cuenca            Fundacion de Cuenca   
3  2012-04-14  Holiday     Local    Libertad      Cantonizacion de Libertad   
4  2012-04-21  Holiday     Local    Riobamba      Cantonizacion de Riobamba   

   transferred  
0        False  
1        

### Filling Null Values

In [8]:
print(oil.isna().sum(), holidays.isna().sum(),stores.isna().sum(), transactions.isna().sum())

date           0
dcoilwtico    43
dtype: int64 date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64 store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64 date            0
store_nbr       0
transactions    0
dtype: int64


In [9]:
oil['dcoilwtico']=oil['dcoilwtico'].ffill()
print(oil.isna().sum())
oil['dcoilwtico']=oil['dcoilwtico'].bfill()
print(oil.isna().sum())

date          0
dcoilwtico    1
dtype: int64
date          0
dcoilwtico    0
dtype: int64


In [10]:
print(oil.head())

         date  dcoilwtico
0  2013-01-01       93.14
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20


## Merging all datasets into Train Dataset

In [11]:
train=train.merge(stores,on="store_nbr",how='left')
test=test.merge(stores,on="store_nbr",how='left')
print(train.head())

   id        date  store_nbr      family  sales  onpromotion   city  \
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0  Quito   
1   1  2013-01-01          1   BABY CARE    0.0            0  Quito   
2   2  2013-01-01          1      BEAUTY    0.0            0  Quito   
3   3  2013-01-01          1   BEVERAGES    0.0            0  Quito   
4   4  2013-01-01          1       BOOKS    0.0            0  Quito   

       state type  cluster  
0  Pichincha    D       13  
1  Pichincha    D       13  
2  Pichincha    D       13  
3  Pichincha    D       13  
4  Pichincha    D       13  


In [12]:
train=train.merge(oil,on="date",how='left')
test=test.merge(oil,on="date",how='left')
print(train.head())

   id        date  store_nbr      family  sales  onpromotion   city  \
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0  Quito   
1   1  2013-01-01          1   BABY CARE    0.0            0  Quito   
2   2  2013-01-01          1      BEAUTY    0.0            0  Quito   
3   3  2013-01-01          1   BEVERAGES    0.0            0  Quito   
4   4  2013-01-01          1       BOOKS    0.0            0  Quito   

       state type  cluster  dcoilwtico  
0  Pichincha    D       13       93.14  
1  Pichincha    D       13       93.14  
2  Pichincha    D       13       93.14  
3  Pichincha    D       13       93.14  
4  Pichincha    D       13       93.14  


In [13]:
train['date']=pd.to_datetime(train['date'])
test['date']=pd.to_datetime(test['date'])

## Feature Engineering

## 

In [14]:
# Extracting year, month, day, and week from date for feature engineering
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.weekday
train['is_weekend'] = train['weekday'].isin([5, 6]).astype(int)

test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday
test['is_weekend'] = test['weekday'].isin([5, 6]).astype(int)


In [15]:
# Converting 'date' column in holidays DataFrame to datetime
holidays['date'] = pd.to_datetime(holidays['date'])


In [16]:
# Merging holidays and events data
train = train.merge(holidays[['date', 'type', 'locale', 'locale_name']], on='date', how='left')
test = test.merge(holidays[['date', 'type', 'locale', 'locale_name']], on='date', how='left')

train.head()


,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,year,month,day,weekday,is_weekend,type_y,locale,locale_name
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,2013,1,1,1,0,Holiday,National,Ecuador
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,2013,1,1,1,0,Holiday,National,Ecuador
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,2013,1,1,1,0,Holiday,National,Ecuador
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,2013,1,1,1,0,Holiday,National,Ecuador
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,2013,1,1,1,0,Holiday,National,Ecuador


In [17]:
train['is_holiday'] = train['type_y'].apply(lambda x: 1 if pd.notnull(x) else 0)
test['is_holiday'] = test['type_y'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [18]:
train['type']=train['type_x']
test['type']=test['type_x']

In [19]:
train.tail()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,year,month,day,weekday,is_weekend,type_y,locale,locale_name,is_holiday,type
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,...,2017,8,15,1,0,Holiday,Local,Riobamba,1,B
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,...,2017,8,15,1,0,Holiday,Local,Riobamba,1,B
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,...,2017,8,15,1,0,Holiday,Local,Riobamba,1,B
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,...,2017,8,15,1,0,Holiday,Local,Riobamba,1,B
3054347,3000887,2017-08-15,9,SEAFOOD,16.000,0,Quito,Pichincha,B,6,...,2017,8,15,1,0,Holiday,Local,Riobamba,1,B


In [20]:
# Dropping columns like 'date' and 'id' as they are not useful for prediction
train = train.drop(columns=['type_x','type_y'])
test = test.drop(columns=['type_x','type_y'])



In [21]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,cluster,dcoilwtico,year,month,day,weekday,is_weekend,locale,locale_name,is_holiday,type
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,13,93.14,2013,1,1,1,0,National,Ecuador,1,D
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,13,93.14,2013,1,1,1,0,National,Ecuador,1,D
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,13,93.14,2013,1,1,1,0,National,Ecuador,1,D
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,13,93.14,2013,1,1,1,0,National,Ecuador,1,D
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,13,93.14,2013,1,1,1,0,National,Ecuador,1,D


In [22]:
# Dropping columns like 'date' and 'id' as they are not useful for prediction
X_train = train.drop(columns=['sales', 'date', 'id', 'type', 'locale', 'locale_name'])
X_test = test.drop(columns=['date', 'id', 'type', 'locale', 'locale_name'])
y_train = train['sales']


## One hot Encoding Features

In [23]:
# Use One-Hot Encoding for categorical features
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)


In [24]:
X_train.head()

,store_nbr,onpromotion,cluster,dcoilwtico,year,month,day,weekday,is_weekend,is_holiday,...,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua
0,1,0,13,93.14,2013,1,1,1,0,1,...,False,False,False,False,False,False,True,False,False,False
1,1,0,13,93.14,2013,1,1,1,0,1,...,False,False,False,False,False,False,True,False,False,False
2,1,0,13,93.14,2013,1,1,1,0,1,...,False,False,False,False,False,False,True,False,False,False
3,1,0,13,93.14,2013,1,1,1,0,1,...,False,False,False,False,False,False,True,False,False,False
4,1,0,13,93.14,2013,1,1,1,0,1,...,False,False,False,False,False,False,True,False,False,False


In [25]:
X_test.head()

,store_nbr,onpromotion,cluster,dcoilwtico,year,month,day,weekday,is_weekend,is_holiday,...,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua
0,1,0,13,46.8,2017,8,16,2,0,0,...,False,False,False,False,False,False,True,False,False,False
1,1,0,13,46.8,2017,8,16,2,0,0,...,False,False,False,False,False,False,True,False,False,False
2,1,2,13,46.8,2017,8,16,2,0,0,...,False,False,False,False,False,False,True,False,False,False
3,1,20,13,46.8,2017,8,16,2,0,0,...,False,False,False,False,False,False,True,False,False,False
4,1,0,13,46.8,2017,8,16,2,0,0,...,False,False,False,False,False,False,True,False,False,False


## Model Selection and Training

In [31]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 1. Splitting training data into train and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# 2. Defining the model with GPU acceleration
model = xgb.XGBRegressor(
    n_estimators=20000,
    learning_rate=0.05,
    max_depth=8,
    n_jobs=-1,
    tree_method='gpu_hist',        # Use GPU for training
    predictor='gpu_predictor',     # Use GPU for prediction
    verbosity=1
)

# 3. Fitting the model with early stopping based on validation loss
model.fit(
    X_train_split, y_train_split,
    early_stopping_rounds=50,
    eval_set=[(X_val, y_val)],
    verbose=True
)


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [16:12:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [16:12:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:1048.21061
[1]	validation_0-rmse:1005.52174
[2]	validation_0-rmse:965.01131
[3]	validation_0-rmse:926.98736
[4]	validation_0-rmse:890.80889
[5]	validation_0-rmse:856.49213
[6]	validation_0-rmse:824.26638
[7]	validation_0-rmse:793.97210
[8]	validation_0-rmse:765.38228
[9]	validation_0-rmse:738.50466
[10]	validation_0-rmse:713.17964
[11]	validation_0-rmse:689.71890
[12]	validation_0-rmse:667.31924
[13]	validation_0-rmse:646.12482
[14]	validation_0-rmse:626.53276
[15]	validation_0-rmse:608.00726
[16]	validation_0-rmse:590.60843
[17]	validation_0-rmse:574.10719
[18]	validation_0-rmse:559.02558
[19]	validation_0-rmse:544.89357
[20]	validation_0-rmse:531.68093
[21]	validation_0-rmse:519.24362
[22]	validation_0-rmse:507.81555
[23]	validation_0-rmse:496.81628
[24]	validation_0-rmse:486.54645
[25]	validation_0-rmse:477.26544
[26]	validation_0-rmse:468.00465
[27]	validation_0-rmse:459.80402
[28]	validation_0-rmse:452.11570
[29]	validation_0-rmse:444.92147
[30]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=20000, n_jobs=-1,
             num_parallel_tree=None, predictor='gpu_predictor', ...)

In [35]:
print("Mean sales:", y_train.mean())
print("Std dev of sales:", y_train.std())


Mean sales: 359.0208922513905
Std dev of sales: 1107.2858972310848


In [36]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("Validation RMSE:", rmse)


Validation RMSE: 179.36214469047408


## Model Prediction

In [37]:
import pandas as pd

# predictions on the test dataset
y_test_pred = model.predict(X_test) 


submission = pd.DataFrame({
    'id': test['id'],  # Ensure the test set has the 'id' column
    'sales': y_test_pred
})

# Saving the submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")


Submission file created: submission.csv
